# Regulier Tag Onderhoud

In [9]:
%run ./__init__edwh__notebooks.ipynb

Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) 
[GCC 9.4.0]
 * postgresql://postgres@pgpool:5432/backend
Done.


In [ ]:
omgeving_leiden= Tag("Omgeving Leiden")

## Tag alle praktijken gekoppeld aan de scholen uit de leidse omgeving met 'Omgeving&nbsp;Leiden'
Mits ze niet al gekoppeld zijn uiteraard.

Voor het filteren welke organisatie tags relevant zijn, moet een tag voldoen aan twee regels: 
 1. de tag is gemetadateerd met `TOrganisations` ofwel `Tag('e5b80b90-1851-42d9-aebc-87327e87ed09')`
 1. naam van de slug eindigt op `-<gemeentenaam>`. 
 
Gemeenten zijn: 
 * leiden
 * zoeterwoude
 * voorschoten
 * oegstgeest
 * leiderdorp
 
 Het volgende aantal organisaties valt hieronder:

In [ ]:
%%sql
select count(*)
  from tag 
 where (slug like  '%-leiden'
        or slug like '%-zoeterwoude'
        or slug like '%-voorschoten'
        or slug like '%-oegstgeest'
        or slug like '%-leiderdorp'
      ) and meta_tags like '%|e5b80b90-1851-42d9-aebc-87327e87ed09|%'


In [ ]:
# TOrganisations is een globale tag omdat deze gedefinieerd is in __init__edw__notebooks.ipynb
def organisatie_tags_in_gemeenten(*gemeenten):
    slug_query = db.tag.id < 0 #query die niets oplevert 
    for gemeente in gemeenten:
        # school namen hebben 
        slug_query |= db.tag.slug.like(f'%-{gemeente}')
    return db(slug_query & (db.tag.meta_tags.contains(TOrganisations.gid))).select()

In [ ]:
organisatie_tags = organisatie_tags_in_gemeenten('leiden','zoeterwoude','voorschoten','oegstgeest','leiderdorp')
organisatie_tags 

In [ ]:
# Controleer of de items getagged zijn met de "omgeving leiden" tag
for tag in [Tag(_) for _ in organisatie_tags]: 
    items = db(db.item.tags.contains(tag.gid) & (db.item.author != None) & (db.item.platform=='SvS')).select()
    if items:
        print('\n',tag.name)
        for item in items: 
            juist_getagged = omgeving_leiden.gid in item.tags 
            print(f"   [{'X' if juist_getagged else ' '}] {item.name}")

In [ ]:
# Update items die nog niet getagged zijn met de "omgeving leiden" tag
# let op: items kunnen dubbel voorkomen in het lijstje hierboven, want ze kunnen met meerdere organisaties getagged zijn.
for tag in [Tag(_) for _ in organisatie_tags]: 
    items = db(db.item.tags.contains(tag.gid)).select()
    if items:
        print('\n',tag.name)
        for item in items: 
            juist_getagged = omgeving_leiden.gid in item.tags 
            if not juist_getagged:
                print(f' `{item.name}` wordt getagged met TOmgevingLeiden')
                item.update_record(tags=item.tags+[omgeving_leiden.gid])
db.commit()

## Welke items zijn nog niet gekoppeld aan scholen? 


In [37]:
!pip install strsimpy 

     |████████████████████████████████| 45 kB 2.1 MB/s             


In [5]:
!pip install -U attrs > /dev/null 

In [7]:
from attrs import define

@define
class Empty:
    pass

Empty()

In [33]:
from strsimpy.jaro_winkler import JaroWinkler
jw=JaroWinkler()
import re 
from IPython.display import display
import ipywidgets as widgets

@define
class CompareName: 
    org:Tag
    
    def similar_to(self, written_name):
        return jw.similarity(written_name, self.org.name)
    
    def bind_button(self, item_gid):
        btn = widgets.Button(description="Koppel "+self.org.name, layout=widgets.Layout(width='100ex', margin="0px 0px 0px 80px"))
        def bind(btn):
            print(f'bind {self.org.name} to item:{item_gid}')
        btn.on_click(bind)
        return btn 

In [34]:
db.rollback()


# # join mv__item_tags aan mv__itemsearch 
# query = db.mv__item_tags.item_gid == db.mv__itemsearch.gid 
# query &= db.mv__itemsearch.public == True 
query = db.item.gid == db.mv__item_tags.item_gid 
query &= db.item.author == db.user.gid 
query &= ~db.user.email.like('%@roc.nl')
query &= ~db.user.email.like('%@dutveul.nl')
# zoek alleen naar items gekoppeld aan een school. 
query &= ~db.mv__item_tags.tag_gid.belongs([org.gid for org in TOrganisations.children()]) 
items_zonder_school_tag = db(query).select(db.item.short_description, db.mv__item_tags.item_gid, distinct=db.mv__item_tags.item_gid)
for row in items_zonder_school_tag[:5]:
    teaser = row.item.short_description.split('. ')[0]
    print(row.mv__item_tags.item_gid, teaser[:150])
    links = re.findall(r'\[(.*?)\]\((.*?)\)',teaser)
    print(links)
    if not links: 
        continue 
    written_name = links[0][0] # first hit, first element from the tuple
    matches = sorted([CompareName(org) for org in TOrganisations.children()], key=lambda cn: cn.similar_to(written_name), reverse=True)
    print('\tbest matches:')
    for m in matches[:5]:
        display(m.bind_button(row.mv__item_tags.item_gid))
    print()
    

000543fd-cfb7-486d-9b7e-620177d6b9a5 Op [De Nieuwste School](https://denieuwsteschool.nl/) in Tilburg wordt sinds 2015 gewerkt aan vakoverstijgende vaardigheden, aan de hand van thema's
[('De Nieuwste School', 'https://denieuwsteschool.nl/')]
	best matches:


Button(description='Koppel De Nieuwste School voor vwo havo mavo TILBURG', layout=Layout(margin='0px 0px 0px 8…

Button(description='Koppel De Nieuwe Wiel LOPIK', layout=Layout(margin='0px 0px 0px 80px', width='100ex'), sty…

Button(description='Koppel De Nieuwe School Basisschool EDAM', layout=Layout(margin='0px 0px 0px 80px', width=…

Button(description='Koppel De Vrije School Widar DELFT', layout=Layout(margin='0px 0px 0px 80px', width='100ex…

...

Button(description='Koppel MaxX NEEDE', layout=Layout(margin='0px 0px 0px 80px', width='100ex'), style=ButtonS…

Button(description='Koppel Tuin 17 HETEREN', layout=Layout(margin='0px 0px 0px 80px', width='100ex'), style=Bu…

Button(description='Koppel ROC A12 EDE GLD', layout=Layout(margin='0px 0px 0px 80px', width='100ex'), style=Bu…

Button(description='Koppel DOK12 AMERSFOORT', layout=Layout(margin='0px 0px 0px 80px', width='100ex'), style=B…


00e0ec90-7ba5-4265-90f0-747b944963df Joke Middelbeek heeft ruim 40 jaar haar bijdrage geleverd in het onderwijs
[]


In [13]:
len(TOrganisations.children())

7433